In [378]:
import numpy as np
import pandas as pd
from utils import *
import random
import re
import keras.utils
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.models import load_model
from pickle import dump, load
import keras.backend as K

In [90]:

data = pd.read_excel("boysname.xls")


In [91]:
data = data['BoysName']

In [92]:
print (len(data))

2468


In [93]:
def splitword(word):
    return list(word.lower())

In [94]:
data = data.apply(splitword)

In [95]:
data.head(5)

0    [a, a, c, h, m, a, n,  ]
1       [a, a, d, e, s, h,  ]
2                [a, a, d, i]
3    [a, a, f, r, e, e, n,  ]
4          [a, a, k, a, r,  ]
Name: BoysName, dtype: object

In [97]:
def removeduplicatenames(wordlist):
    chars = []
    for character in wordlist:
        chars.append(character)
        if character == ' ':
            break
    return chars

In [98]:
data = data.apply(removeduplicatenames)

In [99]:


def uniquechars(data):
    chars = []
    for wordlist in list(data):
        for character in wordlist:
            chars.append(character)
            if character == ' ':
                break
    return set(chars)


def seqlength(data):
    seqlength = []
    for wordlist in list(data):
        seqlength.append(len(wordlist))
        if len(wordlist) > 15:
            print (wordlist)
    return seqlength

chars = uniquechars(data)
seqlength = seqlength(data)

['c', 'h', 'i', 'n', 't', 'h', 'a', 'n', 'a', 'i', 'c', 'h', 'e', 'l', 'v', 'a', 'n']
['s', 'h', 'i', 'r', 's', 'h', 'i', 'r', 'c', 'h', 'a', 'n', 'd', 'r', 'a', ' ']


In [100]:
print (max(seqlength))

17


In [101]:
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

There are 2468 total characters and 27 unique characters in your data.


In [102]:
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print(ix_to_char)


{0: ' ', 1: ',', 2: 'a', 3: 'b', 4: 'c', 5: 'd', 6: 'e', 7: 'f', 8: 'g', 9: 'h', 10: 'i', 11: 'j', 12: 'k', 13: 'l', 14: 'm', 15: 'n', 16: 'o', 17: 'p', 18: 'q', 19: 'r', 20: 's', 21: 't', 22: 'u', 23: 'v', 24: 'w', 25: 'y', 26: 'z'}


In [103]:
print (char_to_ix[' '])

0


In [289]:
def name_to_vec(wordlist):
#     print (wordlist)
    v = np.zeros(17, dtype=int)
   
    null_idx = char_to_ix [' ']
    v.fill(null_idx)
#     print (v)
    i = 0
    
    for charvalue in wordlist:
        v[i] = char_to_ix [charvalue]
        i = i + 1
        
    return v

In [290]:
name_to_vec('nate')

array([15,  2, 21,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [106]:
data = data.apply(name_to_vec)

In [107]:
data.head(5)

0    [2, 2, 4, 9, 14, 2, 15, 0, 0, 0, 0, 0, 0, 0, 0...
1    [2, 2, 5, 6, 20, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
2    [2, 2, 5, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
3    [2, 2, 7, 19, 6, 6, 15, 0, 0, 0, 0, 0, 0, 0, 0...
4    [2, 2, 12, 2, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
Name: BoysName, dtype: object

In [108]:
print (type(data))

<class 'pandas.core.series.Series'>


In [135]:
y = data

In [136]:
print (y.shape)

(2468,)


In [137]:
print (type(y))

<class 'pandas.core.series.Series'>


In [138]:
print (y.shape)

(2468,)


In [139]:
y.head(5)

0    [2, 2, 4, 9, 14, 2, 15, 0, 0, 0, 0, 0, 0, 0, 0...
1    [2, 2, 5, 6, 20, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
2    [2, 2, 5, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
3    [2, 2, 7, 19, 6, 6, 15, 0, 0, 0, 0, 0, 0, 0, 0...
4    [2, 2, 12, 2, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
Name: BoysName, dtype: object

In [140]:
def shiftoleft(wordlist,charvalue=char_to_ix[' ']):
    wordlist = wordlist[1:]
    wordlist = np.append(wordlist,charvalue)
    return wordlist
    

In [141]:
y.apply(shiftoleft)

0       [2, 4, 9, 14, 2, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0...
1       [2, 5, 6, 20, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
2       [2, 5, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
3       [2, 7, 19, 6, 6, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0...
4       [2, 12, 2, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
5       [2, 12, 2, 20, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
6       [2, 13, 2, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
7       [2, 15, 5, 2, 13, 6, 6, 3, 0, 0, 0, 0, 0, 0, 0...
8       [2, 20, 9, 10, 20, 9, 0, 0, 0, 0, 0, 0, 0, 0, ...
9       [3, 6, 6, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
10      [3, 9, 2, 25, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
11      [3, 9, 2, 25, 2, 15, 2, 15, 5, 2, 0, 0, 0, 0, ...
12      [3, 9, 2, 25, 2, 17, 19, 2, 5, 2, 0, 0, 0, 0, ...
13      [3, 9, 6, 6, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
14      [3, 9, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
15      [3, 9, 10, 3, 9, 2, 23, 2, 0, 0, 0, 0, 0, 0, 0...
16      [3, 9, 10, 4, 2, 15, 5, 19, 2, 0, 0, 0, 0, 0, ...
17      [3, 9,

In [142]:
print (y[0])

[ 2  2  4  9 14  2 15  0  0  0  0  0  0  0  0  0  0]


In [143]:
print (data[0])

[ 2  2  4  9 14  2 15  0  0  0  0  0  0  0  0  0  0]


In [144]:
print (data[0])

[ 2  2  4  9 14  2 15  0  0  0  0  0  0  0  0  0  0]


In [145]:
print (type(y))

<class 'pandas.core.series.Series'>


In [146]:
print (y.shape)

(2468,)


In [147]:
y = np.concatenate(y).reshape(2468,17,1)

In [148]:
print (y.shape)

(2468, 17, 1)


In [154]:

# one hot encode outputs
y = keras.utils.to_categorical(y, num_classes=vocab_size)

In [149]:
x = data
x = np.concatenate(x).reshape(2468,17,1)

In [150]:
print (x.shape)

(2468, 17, 1)


In [163]:
# define model
model = Sequential()
model.add(LSTM(50,return_sequences=True, input_shape=(17,1)))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 17, 50)            10400     
_________________________________________________________________
dense_1 (Dense)              (None, 17, 27)            1377      
Total params: 11,777
Trainable params: 11,777
Non-trainable params: 0
_________________________________________________________________
None


In [164]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [167]:
model.fit(x, y, epochs=100, verbose=2)

Epoch 1/100
 - 1s - loss: 0.6646 - acc: 0.8025
Epoch 2/100
 - 1s - loss: 0.6453 - acc: 0.8091
Epoch 3/100
 - 1s - loss: 0.6274 - acc: 0.8146
Epoch 4/100
 - 1s - loss: 0.6110 - acc: 0.8197
Epoch 5/100
 - 1s - loss: 0.5953 - acc: 0.8250
Epoch 6/100
 - 1s - loss: 0.5806 - acc: 0.8285
Epoch 7/100
 - 1s - loss: 0.5670 - acc: 0.8318
Epoch 8/100
 - 1s - loss: 0.5539 - acc: 0.8318
Epoch 9/100
 - 1s - loss: 0.5416 - acc: 0.8332
Epoch 10/100
 - 1s - loss: 0.5297 - acc: 0.8354
Epoch 11/100
 - 1s - loss: 0.5184 - acc: 0.8349
Epoch 12/100
 - 1s - loss: 0.5076 - acc: 0.8372
Epoch 13/100
 - 1s - loss: 0.4974 - acc: 0.8393
Epoch 14/100
 - 1s - loss: 0.4873 - acc: 0.8420
Epoch 15/100
 - 1s - loss: 0.4776 - acc: 0.8434
Epoch 16/100
 - 1s - loss: 0.4683 - acc: 0.8483
Epoch 17/100
 - 1s - loss: 0.4595 - acc: 0.8522
Epoch 18/100
 - 1s - loss: 0.4509 - acc: 0.8585
Epoch 19/100
 - 1s - loss: 0.4428 - acc: 0.8602
Epoch 20/100
 - 1s - loss: 0.4350 - acc: 0.8634
Epoch 21/100
 - 1s - loss: 0.4274 - acc: 0.8646
E

In [171]:

model.save('model.h5')

In [176]:
print (char_to_ix)

{' ': 0, ',': 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'y': 25, 'z': 26}


In [179]:
dump(char_to_ix, open('mapping.pkl', 'wb'))

In [419]:
import tensorflow as tf
sess = tf.Session()

from keras import backend as K
K.set_session(sess)

# load the model
model = load_model('model.h5')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [420]:

2
# load the mapping
mapping = load(open('mapping.pkl', 'rb'))

In [421]:
in_text = 'i'
# encode the characters as integers
encoded = [mapping[char] for char in in_text]

In [422]:
print (encoded)

[10]


In [423]:
print (mapping)

{' ': 0, ',': 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'y': 25, 'z': 26}


In [441]:
# text = np.array([1,2,3,4,5,0,0,0,0,0,0,0,0,0,0,0,0])

text = np.random.randint(1, high=26, size=6, dtype='l')
remaining = np.array([0,0,0,0,0,0,0,0,0,0,0])
text = np.concatenate((text, remaining))

In [442]:
print (text)

[11 14  1  5 22 18  0  0  0  0  0  0  0  0  0  0  0]


In [443]:
print (len(text))

17


In [444]:
# print (type((data[0])))

In [445]:
print (text.shape)

(17,)


In [446]:
print (text)

[11 14  1  5 22 18  0  0  0  0  0  0  0  0  0  0  0]


In [447]:
actualchars = []
for value in text:
    actualchars.append(list(mapping.keys())[list(mapping.values()).index(value)])
print (actualchars)

['j', 'm', ',', 'd', 'u', 'q', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']


In [448]:

text  = text.reshape(1,17,1)

In [449]:
 print (text)

[[[11]
  [14]
  [ 1]
  [ 5]
  [22]
  [18]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]]]


In [450]:
np.save('text.npy', text)

In [451]:
%pwd

'/Users/jac812i/Documents/projects/indianbabynames'

In [433]:
prediction = model.predict(text)

In [434]:
# print (prediction)

In [435]:
print (prediction.shape)

(1, 17, 27)


In [436]:
i = 0
predictedvalues = []
for values in prediction:
#     print (value)
    for value in values:
        predictedvalues.append(np.argmax(value))
        i = i + 1
print (predictedvalues)
print (len(predictedvalues))

[17, 14, 22, 20, 25, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
17


In [437]:
print (mapping)

{' ': 0, ',': 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'y': 25, 'z': 26}


In [438]:
predictedchars = []
for value in predictedvalues:
    predictedchars.append(list(mapping.keys())[list(mapping.values()).index(value)])
print (predictedchars)

['p', 'm', 'u', 's', 'y', 'n', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
